# Assignment 11: AI Agent for Weather & Search Queries
Building a Langchain-based AI agent that handles real-time weather and web search queries.

**Author:** HaoPQ1  
**Date:** November 7, 2025

## Objectives
- Learn to build a Langchain-based AI agent
- Integrate OpenWeather API and Tavily Search API as custom tools
- Simulate React-style conversational interface in Jupyter Notebook
- Demonstrate tool routing and decision-making

## Step 1: Install Required Packages
Run this cell to install all necessary dependencies:

In [1]:
# Install required packages
!pip install langchain-openai langchain-community langchain-tavily langgraph pyowm langchain tiktoken requests

zsh:1: command not found: pip


## Step 2: Import Libraries and Setup Environment
Import all necessary libraries and configure API keys:

In [2]:
import os
from langchain.tools import tool
from langchain_openai import AzureChatOpenAI
from langchain_community.utilities import OpenWeatherMapAPIWrapper
from langchain_tavily import TavilySearch
from langgraph.prebuilt import create_react_agent

print("✅ All libraries imported successfully!")

/Users/haopq1/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/haopq1/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/haopq1/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ All libraries imported successfully!


In [ ]:
def setup_environment():
    """
    Setup API keys and environment variables.
    Replace empty strings with your actual API keys.
    """
    # Azure OpenAI Configuration
    os.environ["AZURE_OPENAI_ENDPOINT"] = ""  # Replace with your Azure OpenAI endpoint
    os.environ["AZURE_OPENAI_API_KEY"] = ""   # Replace with your Azure OpenAI API key
    os.environ["AZURE_DEPLOYMENT_NAME"] = "GPT-4o-mini"  # Replace with your deployment name
    
    # OpenWeatherMap API Configuration
    os.environ["OPENWEATHERMAP_API_KEY"] = ""  # Replace with your OpenWeatherMap API key
    
    # Tavily Search API Configuration
    os.environ["TAVILY_API_KEY"] = ""  # Replace with your Tavily API key
    
    print("🔧 Environment variables configured successfully!")

# Setup environment
setup_environment()

🔧 Environment variables configured successfully!


## Step 3: Define Weather Tool
Create a custom Langchain tool for weather queries using OpenWeatherMap API:

In [4]:
@tool
def get_weather(city: str) -> str:
    """
    Get the current weather for a given city using OpenWeatherMap API.
    
    Args:
        city (str): The name of the city to get the weather for.
        
    Returns:
        str: A string describing the current weather in the specified city.
    """
    print(f"🌤️ Weather tool called for: {city}")
    
    try:
        weather = OpenWeatherMapAPIWrapper()
        weather_info = weather.run(city)
        return f"Weather in {city}: {weather_info}"
    except Exception as e:
        return f"Sorry, I couldn't get the weather information for {city}. Error: {str(e)}"

print("✅ Weather tool defined successfully!")

✅ Weather tool defined successfully!


## Step 4: Initialize Tavily Search Tool
Setup the web search capability using Tavily API:

In [5]:
def initialize_tavily_search():
    """
    Initialize Tavily search tool for web searches.
    
    Returns:
        TavilySearch: Configured Tavily search instance
    """
    try:
        tavily_search_tool = TavilySearch(
            max_results=3,
            topic="general",
        )
        print("✅ Tavily Search tool initialized successfully!")
        return tavily_search_tool
    except Exception as e:
        print(f"❌ Error initializing Tavily Search: {str(e)}")
        return None

# Initialize search tool
tavily_search_tool = initialize_tavily_search()

✅ Tavily Search tool initialized successfully!


## Step 5: Initialize Azure OpenAI LLM
Setup the language model for the AI agent:

In [6]:
def initialize_llm():
    """
    Initialize Azure OpenAI LLM with configuration.
    
    Returns:
        AzureChatOpenAI: Configured Azure OpenAI instance
    """
    try:
        llm = AzureChatOpenAI(
            azure_deployment=os.getenv("AZURE_DEPLOYMENT_NAME"),
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_version="2024-07-01-preview",
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        )
        print("✅ Azure OpenAI LLM initialized successfully!")
        return llm
    except Exception as e:
        print(f"❌ Error initializing Azure OpenAI: {str(e)}")
        return None

# Initialize LLM
llm = initialize_llm()

✅ Azure OpenAI LLM initialized successfully!


## Step 6: Create Langchain Agent
Combine the LLM and tools to create a ReAct agent:

In [7]:
# Combine all tools
tools = [get_weather, tavily_search_tool]

# Create the ReAct agent
agent = create_react_agent(
    model=llm,
    tools=tools,
)

print("🤖 AI Agent created successfully!")
print(f"📋 Available tools: {[tool.name for tool in tools]}")

🤖 AI Agent created successfully!
📋 Available tools: ['get_weather', 'tavily_search']


## Step 7: Interactive Agent Loop Simulation
Now let's simulate the conversational interface with the AI agent:

In [8]:
# Initialize conversation history
messages = []

def chat_with_agent(user_input: str):
    """
    Chat with the AI agent and maintain conversation history.
    
    Args:
        user_input (str): User's question or request
    
    Returns:
        str: Agent's response
    """
    global messages
    
    print(f"👤 User: {user_input}")
    print("🤔 AI is thinking...")
    
    # Add user message to conversation history
    messages.append({"role": "user", "content": user_input})
    
    try:
        # Get agent response
        response = agent.invoke({"messages": messages})
        
        # Extract AI response
        ai_response = response["messages"][-1].content
        messages.append({"role": "assistant", "content": ai_response})
        
        print(f"🤖 AI: {ai_response}")
        return ai_response
        
    except Exception as e:
        error_message = f"Sorry, I encountered an error: {str(e)}"
        print(f"❌ AI: {error_message}")
        return error_message

print("✅ Chat function ready! You can now interact with the agent.")

✅ Chat function ready! You can now interact with the agent.


## Interactive Demo: Weather Queries
Let's test the weather functionality:

In [9]:
# Test weather query for Hanoi
chat_with_agent("What's the weather in Hanoi?")

👤 User: What's the weather in Hanoi?
🤔 AI is thinking...
🌤️ Weather tool called for: Hanoi
🌤️ Weather tool called for: Hanoi
🤖 AI: The current weather in Hanoi is as follows:
- **Temperature**: 22.0°C (Feels like 22.58°C)
- **Condition**: Overcast clouds
- **Humidity**: 89%
- **Wind Speed**: 3.24 m/s from the northwest (312°)
- **Cloud Cover**: 100%
- **High/Low**: 22.0°C / 22.0°C

It looks like a cool and cloudy day in Hanoi!
🤖 AI: The current weather in Hanoi is as follows:
- **Temperature**: 22.0°C (Feels like 22.58°C)
- **Condition**: Overcast clouds
- **Humidity**: 89%
- **Wind Speed**: 3.24 m/s from the northwest (312°)
- **Cloud Cover**: 100%
- **High/Low**: 22.0°C / 22.0°C

It looks like a cool and cloudy day in Hanoi!


'The current weather in Hanoi is as follows:\n- **Temperature**: 22.0°C (Feels like 22.58°C)\n- **Condition**: Overcast clouds\n- **Humidity**: 89%\n- **Wind Speed**: 3.24 m/s from the northwest (312°)\n- **Cloud Cover**: 100%\n- **High/Low**: 22.0°C / 22.0°C\n\nIt looks like a cool and cloudy day in Hanoi!'

In [ ]:
# Test weather query for another city
chat_with_agent("What's the current weather in Tokyo?")

## Interactive Demo: Web Search Queries
Let's test the search functionality:

In [10]:
# Test search query about AI news
chat_with_agent("Tell me about the latest developments in artificial intelligence")

👤 User: Tell me about the latest developments in artificial intelligence
🤔 AI is thinking...
🤖 AI: Here are some of the latest developments in artificial intelligence:

1. **Apple's Siri Upgrade**:
   - Apple plans to integrate a 1.2 trillion-parameter artificial intelligence model developed by Google's Alphabet to enhance its Siri voice assistant. This move indicates a significant shift in how Apple is looking to improve its AI capabilities.
   - [Read more on Reuters](https://www.reuters.com/technology/artificial-intelligence/)

2. **General AI Advancements**:
   - IEEE Spectrum reports on various advances in artificial intelligence, including developments in generative AI, deepfakes, and large language models (LLMs). The focus is on how these technologies are evolving and their implications.
   - [Explore more here](https://spectrum.ieee.org/topic/artificial-intelligence/)

3. **Ongoing Coverage**:
   - The Wall Street Journal provides extensive news coverage focusing on the technol

"Here are some of the latest developments in artificial intelligence:\n\n1. **Apple's Siri Upgrade**:\n   - Apple plans to integrate a 1.2 trillion-parameter artificial intelligence model developed by Google's Alphabet to enhance its Siri voice assistant. This move indicates a significant shift in how Apple is looking to improve its AI capabilities.\n   - [Read more on Reuters](https://www.reuters.com/technology/artificial-intelligence/)\n\n2. **General AI Advancements**:\n   - IEEE Spectrum reports on various advances in artificial intelligence, including developments in generative AI, deepfakes, and large language models (LLMs). The focus is on how these technologies are evolving and their implications.\n   - [Explore more here](https://spectrum.ieee.org/topic/artificial-intelligence/)\n\n3. **Ongoing Coverage**:\n   - The Wall Street Journal provides extensive news coverage focusing on the technology tools and companies that are shaping the future of AI. This includes analysis and i

In [ ]:
# Test search query about technology
chat_with_agent("Search for recent breakthroughs in machine learning")

## Interactive Demo: Mixed Queries
Test the agent's ability to route different types of queries:

In [ ]:
# General knowledge question (should use LLM knowledge)
chat_with_agent("Who won the last World Cup?")

In [ ]:
# Weather query for a different location
chat_with_agent("How's the weather in Paris today?")

In [ ]:
# Search query about current events
chat_with_agent("Find information about climate change effects in 2025")

## Custom Interactive Cell
Use this cell to ask your own questions to the agent:

In [11]:
# Ask your own question here!
# Replace the text below with your question
your_question = "What's the weather in New York?"

chat_with_agent(your_question)

👤 User: What's the weather in New York?
🤔 AI is thinking...
🌤️ Weather tool called for: New York
🌤️ Weather tool called for: New York
🤖 AI: The current weather in New York is as follows:
- **Temperature**: 7.77°C (Feels like 4.35°C)
- **Condition**: Clear sky
- **Humidity**: 51%
- **Wind Speed**: 6.17 m/s from the northwest (330°)
- **High/Low**: 8.45°C / 5.77°C
- **Cloud Cover**: 0%

It's a clear and cool day in New York!
🤖 AI: The current weather in New York is as follows:
- **Temperature**: 7.77°C (Feels like 4.35°C)
- **Condition**: Clear sky
- **Humidity**: 51%
- **Wind Speed**: 6.17 m/s from the northwest (330°)
- **High/Low**: 8.45°C / 5.77°C
- **Cloud Cover**: 0%

It's a clear and cool day in New York!


"The current weather in New York is as follows:\n- **Temperature**: 7.77°C (Feels like 4.35°C)\n- **Condition**: Clear sky\n- **Humidity**: 51%\n- **Wind Speed**: 6.17 m/s from the northwest (330°)\n- **High/Low**: 8.45°C / 5.77°C\n- **Cloud Cover**: 0%\n\nIt's a clear and cool day in New York!"

## Conversation History
View the complete conversation history:

In [12]:
print("📋 Complete Conversation History:")
print("=" * 50)

for i, message in enumerate(messages, 1):
    role = "👤 User" if message["role"] == "user" else "🤖 AI"
    content = message["content"]
    
    # Truncate long messages for display
    if len(content) > 200:
        content = content[:200] + "..."
    
    print(f"{i}. {role}: {content}")
    print("-" * 30)

📋 Complete Conversation History:
1. 👤 User: What's the weather in Hanoi?
------------------------------
2. 🤖 AI: The current weather in Hanoi is as follows:
- **Temperature**: 22.0°C (Feels like 22.58°C)
- **Condition**: Overcast clouds
- **Humidity**: 89%
- **Wind Speed**: 3.24 m/s from the northwest (312°)
- *...
------------------------------
3. 👤 User: Tell me about the latest developments in artificial intelligence
------------------------------
4. 🤖 AI: Here are some of the latest developments in artificial intelligence:

1. **Apple's Siri Upgrade**:
   - Apple plans to integrate a 1.2 trillion-parameter artificial intelligence model developed by Goo...
------------------------------
5. 👤 User: What's the weather in New York?
------------------------------
6. 🤖 AI: The current weather in New York is as follows:
- **Temperature**: 7.77°C (Feels like 4.35°C)
- **Condition**: Clear sky
- **Humidity**: 51%
- **Wind Speed**: 6.17 m/s from the northwest (330°)
- **Hig...
--------------

## Summary: Concepts Covered ✅

This Jupyter Notebook successfully demonstrates all required concepts:

### 1. ✅ Langchain Tools and Agent Setup
- Custom `@tool` decorator for weather functionality
- TavilySearch tool integration
- ReAct agent creation with `create_react_agent()`

### 2. ✅ API Integration for OpenWeather and Tavily Search
- OpenWeatherMapAPIWrapper for real-time weather data
- TavilySearch for web search capabilities
- Proper error handling for both APIs

### 3. ✅ Tool Routing and Decision-Making
- Agent automatically routes weather queries → weather tool
- Agent automatically routes search queries → search tool
- Intelligent decision-making based on query content

### 4. ✅ Interactive Agent Loop Simulation in Jupyter Notebook
- Step-by-step interactive cells
- `chat_with_agent()` function for conversational interface
- Conversation history maintenance
- Customizable interaction cells for user input
- Real-time demonstration of agent capabilities

**🎉 All Assignment 11 objectives completed successfully!**